# Bayesian network

This is a bayesian network made to model the relationships between 

In [3]:
# Import relevant packages
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
import sys

In [36]:
# We first create a model which containts edges of the graph
model = BayesianModel([('Smoking', 'YellowFingers'), ('Smoking', 'Cancer'), ('SolarFlares', 'Radiation'), 
                       ('Microwave', 'Radiation'), ('Radiation', 'Cancer')])

# Prior probability for Smoking P(S)
cpd_S = TabularCPD(variable='Smoking', variable_card=2, values=[[0.7, 0.3]])

# Prior probability for Solar Flares P(F)
cpd_F = TabularCPD(variable='SolarFlares', variable_card=2, values=[[0.2, 0.8]])

# Prior probability for Microwave P(M)
cpd_M = TabularCPD(variable='Microwave', variable_card=2, values=[[0.1, 0.9]])

# Conditional probability for Radiation or P(Radiation| SolarFlares, Microwave)
cpd_R = TabularCPD(variable='Radiation',variable_card=2, values = [[0.9, 0.8, 0.8, 0.1],
                                                               [0.1, 0.2, 0.2, 0.9]],
                                                               evidence = ['SolarFlares', 'Microwave'],
                                                               evidence_card=[2, 2])

# Conditional probability for Cancer or P(Cancer | Smoking, Radiation)
cpd_C = TabularCPD(variable='Cancer',variable_card=2, values = [[0.9, 0.4, 0.7, 0.1],
                                                               [0.1, 0.6, 0.3, 0.9]],
                                                               evidence = ['Smoking', 'Radiation'],
                                                               evidence_card=[2, 2])

# Conditional probability for Yellowfingers or P(YellowFingers | Smoking)
cpd_Y = TabularCPD(variable='YellowFingers', variable_card=2, values = [[0.89, 0.2],
                                                               [0.11, 0.8]],
                                                               evidence = ['Smoking'],
                                                               evidence_card=[2])
# Add cpds to the model
model.add_cpds(cpd_S, cpd_F, cpd_M, cpd_R, cpd_C, cpd_Y)

print(model.check_model())

print(model.get_cpds('Smoking'))
print(model.get_cpds('SolarFlares'))
print(model.get_cpds('Microwave'))
print(model.get_cpds('Radiation'))
print(model.get_cpds('Cancer'))
print(model.get_cpds('YellowFingers'))


from pgmpy.inference import VariableElimination

# Going to do variable elimination
infer = VariableElimination(model)

# Get Marginal probability of Burglary and quake
cancer_query = infer.query(['Cancer'])
cancer_factor = cancer_query['Cancer']
print('Prior Probability of Cancer')
print(cancer_factor)

# Compute probability of Smoking given Cancer
smoking_query = infer.query(['Smoking'], evidence={'Cancer':1})
print('Probability of smoking given the person has cancer')
smoking_factor = smoking_query['Smoking']
print(smoking_factor)

#Get independencies
#model.local_independencies('SolarFlares')
#model.get_independencies()

# Compute probability of Cancer given Microwave=0
noMicrowave_query = infer.query(['Cancer'], evidence={'Microwave':0})
print('Probability of cancer given the person has never used a microwave')
noMicrowave_factor = noMicrowave_query['Cancer']
print(noMicrowave_factor)

True
╒═══════════╤═════╕
│ Smoking_0 │ 0.7 │
├───────────┼─────┤
│ Smoking_1 │ 0.3 │
╘═══════════╧═════╛
╒═══════════════╤═════╕
│ SolarFlares_0 │ 0.2 │
├───────────────┼─────┤
│ SolarFlares_1 │ 0.8 │
╘═══════════════╧═════╛
╒═════════════╤═════╕
│ Microwave_0 │ 0.1 │
├─────────────┼─────┤
│ Microwave_1 │ 0.9 │
╘═════════════╧═════╛
╒═════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╕
│ SolarFlares │ SolarFlares_0 │ SolarFlares_0 │ SolarFlares_1 │ SolarFlares_1 │
├─────────────┼───────────────┼───────────────┼───────────────┼───────────────┤
│ Microwave   │ Microwave_0   │ Microwave_1   │ Microwave_0   │ Microwave_1   │
├─────────────┼───────────────┼───────────────┼───────────────┼───────────────┤
│ Radiation_0 │ 0.9           │ 0.8           │ 0.8           │ 0.1           │
├─────────────┼───────────────┼───────────────┼───────────────┼───────────────┤
│ Radiation_1 │ 0.1           │ 0.2           │ 0.2           │ 0.9           │
╘═════════════╧══════════